In [2]:
import pandas as pd
import sklearn
import seaborn as sns
import numpy as np

In [3]:
!pip install pyarrow

In [4]:
jan_df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
feb_df = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

In [5]:
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

1. There's 19 columns in January

In [6]:
jan_df['duration'] = jan_df['tpep_dropoff_datetime'] - jan_df ['tpep_pickup_datetime']
jan_df['duration'].std()

Timedelta('0 days 00:42:35.661074517')

2. The standard deviation of trip durations is 42.59 minutes

In [7]:
one_min = pd.Timedelta(minutes=1)
one_hour = pd.Timedelta(hours=1)

jan_df = jan_df[(jan_df['duration'] >= one_min) & (jan_df['duration'] <= one_hour)]
jan_df.duration = jan_df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
jan_df['duration'].fillna(-1)

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3009173, dtype: float64

In [9]:
jan_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


3. We started with 3066766 entries, and we ended with 3009173 entries. We have 98% of the data left after outliers.

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [11]:
# Features of our model
categorical = ['PULocationID', 'DOLocationID']
jan_df[categorical] = jan_df[categorical].astype(str)

dv = DictVectorizer()

# Training the model
# .fit_transform is for making a feature model
train_dicts = jan_df[categorical].to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

In [12]:
len(dv.get_feature_names())

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


515

4. The number of columns from this feature matrix is 515

In [13]:
target = 'duration'
y_train = jan_df[target].values

In [14]:
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [15]:
# We train duration on pickuplocation, dropofflocation, and trip_distance
lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)

np.sqrt(mean_squared_error(y_train, y_pred, squared = False))

2.7657297459788754

5. I was not able to get the exact answer, but the closest one is 3.64

In [16]:
# Preprocessing for feb
feb_df['duration'] = feb_df['tpep_dropoff_datetime'] - feb_df['tpep_pickup_datetime']

feb_df = feb_df[(feb_df['duration'] >= one_min) & (feb_df['duration'] <= one_hour)]
feb_df.duration = feb_df.duration.apply(lambda td: td.total_seconds() / 60)

feb_df['duration'].fillna(-1)

feb_df[categorical] = feb_df[categorical].astype(str)

In [17]:
train_val = feb_df[categorical].to_dict(orient = 'records')
x_val = dv.transform(train_val)

y_val = feb_df[target].values

In [18]:
# We train duration on pickuplocation, dropofflocation
lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred2 = lr.predict(x_val)

np.sqrt(mean_squared_error(y_val, y_pred2, squared = False))

2.794965623456649

6. I was not able to get the exact answer, but the closest one is 3.81. In terms of logic, you would also want your RMSE for your validating model to be similar to that of your training model.